<a href="https://colab.research.google.com/github/gkv856/KaggleData/blob/main/Watson_BERT_v001.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf


In [2]:
!pip install transformers

In [3]:
from transformers import BertTokenizer, TFBertModel
import matplotlib.pyplot as plt

In [4]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [235]:
TRAIN_URL = "https://raw.githubusercontent.com/gkv856/KaggleData/main/watson_train.csv"
# TRAIN_URL ="https://raw.githubusercontent.com/gkv856/KaggleData/main/train.csv"
train = pd.read_csv(TRAIN_URL)

In [236]:
train.head()

,id,premise,hypothesis,lang_abv,language,label
0,5130fd2cb5,and these comments were considered in formulat...,The rules developed in the interim were put to...,en,English,0
1,5b72532a0b,These are issues that we wrestle with in pract...,Practice groups are not permitted to work on t...,en,English,2
2,3931fbe82a,Des petites choses comme celles-là font une di...,J'essayais d'accomplir quelque chose.,fr,French,0
3,5622f0c60b,you know they can't really defend themselves l...,They can't defend themselves because of their ...,en,English,0
4,86aaa48b45,ในการเล่นบทบาทสมมุติก็เช่นกัน โอกาสที่จะได้แสด...,เด็กสามารถเห็นได้ว่าชาติพันธุ์แตกต่างกันอย่างไร,th,Thai,1


In [239]:
# later you can use the whole data for traiing and test
# train_df, valid_df = train_test_split(train, 
#                                        random_state=42, 
#                                       #  train_size=0.9, 
#                                        test_size=.25,
#                                        stratify=train[["label", "lang_abv"]].values)

# use the below structure for testing if the data is huge
train_df, remaining = train_test_split(train, 
                                       random_state=42, 
                                       train_size=0.0095, 
                                       stratify=train[["label", "lang_abv"]].values)

valid_df, _ = train_test_split(remaining, 
                              random_state=42, 
                              train_size=0.0095, 
                              stratify=remaining[["label", "lang_abv"]].values)
len(train_df), len(valid_df)

(115, 114)

In [241]:
# creating datasets
with tf.device('/cpu:0'):
  train_data = tf.data.Dataset.from_tensor_slices(((train_df[["premise", "hypothesis"]].values), 
                                                   train_df["label"].values))
  
  # test_data = tf.data.Dataset.from_tensor_slices((valid_df["premise"].values,
  #                                                  valid_df["label"].values))
train_data#, test_data

<TensorSliceDataset shapes: ((2,), ()), types: (tf.string, tf.int64)>

In [242]:
for d, label in train_data.take(4):
    print(d, label)
    print("\n")


tf.Tensor(
[b'\xd8\xb4\xd8\xa7\xda\xa9\xd8\xb1 \xda\xa9\xd9\x88 \xdb\x81\xd8\xb3\xd9\xbe\xd8\xa7\xd9\x86\xd9\x88\xdb\x8c \xd8\xad\xda\xa9\xd8\xa7\xd9\x85 \xd9\x86\xdb\x92 \xd9\x81\xd8\xa7\xd8\xb1\xdb\x8c\xda\x88 \xdb\x81\xd9\x84\xd8\xa7\xd9\x84\xdb\x8c \xda\xa9\xdb\x92 \xd8\xb7\xd9\x88\xd8\xb1 \xd9\xbe\xd8\xb1 \xd8\xb4\xd9\x86\xd8\xa7\xd8\xae\xd8\xaa \xda\xa9\xdb\x8c\xd8\xa7 \xdb\x81\xdb\x92.'
 b'\xd8\xb4\xd8\xa7\xda\xa9\xd8\xb1 \xda\xa9\xd8\xa7  \xd8\xa7\xdb\x8c\xda\xa9 \xd8\xba\xd9\x84\xd8\xb7 \xd9\x86\xd8\xa7\xd9\x85 \xd8\xaa\xda\xbe\xd8\xa7 \xd8\xac\xd9\x88 \xd8\xb3\xd8\xa7\xd9\x84\xd9\x88\xda\xba \xd8\xb3\xdb\x92 \xd8\xba\xdb\x8c\xd8\xb1 \xd9\x85\xd8\xaa\xd9\x88\xd8\xac\xdb\x81 \xd8\xb1\xdb\x81\xd8\xa7.'], shape=(2,), dtype=string) tf.Tensor(1, shape=(), dtype=int64)


tf.Tensor(
[b'\xe0\xa4\x94\xe0\xa4\xb0, \xe0\xa4\xb5\xe0\xa5\x8b \xe0\xa4\xa5\xe0\xa5\x8b\xe0\xa5\x9c\xe0\xa4\xbe \xe0\xa4\x85\xe0\xa4\xb2\xe0\xa4\x97 \xe0\xa4\xb9\xe0\xa5\x88, \xe0\xa4\x9c\xe0\xa5\x88\xe0\xa4\xb8\x

In [243]:
"""
Each line of the dataset is composed of the review text and its label
- Data preprocessing consists of transforming text to BERT input features:
input_word_ids, input_mask, segment_ids
- In the process, tokenizing the text is done with the provided BERT model tokenizer
"""

# Label categories, right now our data has these categories
label_list = [0, 1, 2]

# maximum length of (token) input sequences, or the words in a question
# to save speed we should reset this
max_seq_length = 128

train_batch_size = 32

In [65]:
model_name = 'bert-base-multilingual-cased'
tokenizer = BertTokenizer.from_pretrained(model_name)

In [66]:
def encode_sentence(s, is_cls=False):
   tokens = tokenizer.tokenize(s)
   if not is_cls:
    tokens.append('[SEP]')
   return tokenizer.convert_tokens_to_ids(tokens)

In [73]:
tf.convert_to_tensor(encode_sentence("i love machine learning"), dtype=tf.int32)

<tf.Tensor: shape=(5,), dtype=int32, numpy=array([  177, 16138, 21432, 26901,   102], dtype=int32)>

In [231]:
def pad_tokens(t1, t2, max_seq_length=max_seq_length):
  
  half_len_t1 = max_seq_length // 2 - 1
  half_len_t2 = max_seq_length // 2
  
  frt = [t1[0]]
  lst = [t1[-1]]
  t1 = t1[1:-1]
  t1 = frt +t1[1:half_len_t1-1]  + lst
  # print(t1)

  lst = [t2[-1]]
  t2 = t2[0:-1]
  t2 = t2[1:half_len_t2]  + lst
  # print(t2)


  len_t1 = len(t1)
  len_t2 = len(t2)

  if len_t1 < half_len_t1:
    len_pad = half_len_t1 - len_t1
    lst = [0] * len_pad
    zeros = tf.constant(lst, dtype=tf.int32)
    t1 = tf.concat([t1, zeros], axis=0) 

  if len_t2 < half_len_t2:
    len_pad = half_len_t2 - len_t2
    lst = [0] * len_pad
    zeros = tf.constant(lst, dtype=tf.int32)
    t2 = tf.concat([t2, zeros], axis=0)
  
  return t1, t2

def to_feature(texta, textb, label):#, tokenizer=tokenizer):
  hypothese = texta.numpy().decode('UTF-8')
  premise = textb.numpy().decode('UTF-8')
  
  cls = encode_sentence("[CLS]", is_cls=True)
  
  sentence1 = encode_sentence(hypothese)
  sentence2 = encode_sentence(premise)
  
  sentence1, sentence2  = pad_tokens(sentence1, sentence2, max_seq_length=128)
  

  sentence1 = tf.convert_to_tensor(sentence1, dtype=tf.int32)
  sentence2 = tf.convert_to_tensor(sentence2, dtype=tf.int32)
  

  input_word_ids = tf.concat([cls, sentence1, sentence2], axis=0)
  input_mask = tf.ones_like(input_word_ids, dtype=tf.int32)

  type_cls = tf.zeros_like(cls, dtype=tf.int32)
  type_s1 = tf.zeros_like(sentence1, dtype=tf.int32)
  type_s2 = tf.ones_like(sentence2, dtype=tf.int32)

  input_type_ids = tf.concat([type_cls, type_s1, type_s2], axis=-1) 


  label = tf.cast(label, dtype=tf.int32)


  return (input_word_ids, input_mask, input_type_ids, label)


In [214]:
def to_feature_map(texta, textb, label):
  #     print(text, label)
  out = tf.py_function(to_feature, inp=[texta, textb, label], Tout=[tf.int32, tf.int32, tf.int32, tf.int32])
  
  iids, imask, segids, label = out[0], out[1], out[2], out[3]

 
  x = {
    "input_word_ids": iids,
    "input_mask": imask,
    "input_type_ids": segids
  }
  return (x, label)

In [244]:
t = "1 2 3 4 5 6 7 8 9 10"
# t = ""
x = to_feature_map(t, t, 4 )
x

({'input_mask': <tf.Tensor: shape=(128,), dtype=int32, numpy=
  array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], dtype=int32)>,
  'input_type_ids': <tf.Tensor: shape=(128,), dtype=int32, numpy=
  array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1

In [249]:
s = train_data.take(1)
for t, l in s:
  inps = to_feature_map(t[0], t[1], l)[0]  
  print(inps["input_word_ids"].shape, inps["input_mask"].shape, inps["input_type_ids"].shape)

(128,) (128,) (128,)
